In [4]:
#Date: 9-22-2022
#Author: Talia Kurtz

#The purpose of the code:
#This code is used to generate the frequency, Sammon plots, and the MSLP raw data plots. The sammon plot import is large. This code will loop through the file of SOMs 
#saved in the folder that we generated in Step 2 and plot each one individually and save the outputs. You will be calling in the data_train again and the MSLP Raw data. 

#What does this code save?
#This code saves the plots for each SOM made from the folder defined in Step #2. The frequency, sammon plot, and the MSLP raw composite plots.
###########################################################################################################################################################################
#Imports
#Imports
import xarray as xr
import numpy as np
import pandas as pd
import warnings
from itertools import product
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pickle
import glob as glob
import os
import pickle
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
from numpy import savetxt
from numpy import loadtxt

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
#################################################################################################################################################################
#Functions Used in the Code
def getList(dict):
    list = []
    for key in winmap.keys():
        list.append(key)
        
    return list
def sammon(x, n, display = 2, inputdist = 'raw', maxhalves = 20, maxiter = 500, tolfun = 1e-9, init = 'default'):

    import numpy as np 
    from scipy.spatial.distance import cdist

    """Perform Sammon mapping on dataset x
    y = sammon(x) applies the Sammon nonlinear mapping procedure on
    multivariate data x, where each row represents a pattern and each column
    represents a feature.  On completion, y contains the corresponding
    co-ordinates of each point on the map.  By default, a two-dimensional
    map is created.  Note if x contains any duplicated rows, SAMMON will
    fail (ungracefully). 
    [y,E] = sammon(x) also returns the value of the cost function in E (i.e.
    the stress of the mapping).
    An N-dimensional output map is generated by y = sammon(x,n) .
    A set of optimisation options can be specified using optional
    arguments, y = sammon(x,n,[OPTS]):
       maxiter        - maximum number of iterations
       tolfun         - relative tolerance on objective function
       maxhalves      - maximum number of step halvings
       input          - {'raw','distance'} if set to 'distance', X is 
                        interpreted as a matrix of pairwise distances.
       display        - 0 to 2. 0 least verbose, 2 max verbose.
       init           - {'pca', 'cmdscale', random', 'default'}
                        default is 'pca' if input is 'raw', 
                        'msdcale' if input is 'distance'
    The default options are retrieved by calling sammon(x) with no
    parameters.
    File        : sammon.py
    Date        : 18 April 2014
    Authors     : Tom J. Pollard (tom.pollard.11@ucl.ac.uk)
                : Ported from MATLAB implementation by 
                  Gavin C. Cawley and Nicola L. C. Talbot
    Description : Simple python implementation of Sammon's non-linear
                  mapping algorithm [1].
    References  : [1] Sammon, John W. Jr., "A Nonlinear Mapping for Data
                  Structure Analysis", IEEE Transactions on Computers,
                  vol. C-18, no. 5, pp 401-409, May 1969.
    Copyright   : (c) Dr Gavin C. Cawley, November 2007.
    This program is free software; you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation; either version 2 of the License, or
    (at your option) any later version.
    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.
    You should have received a copy of the GNU General Public License
    along with this program; if not, write to the Free Software
    Foundation, Inc., 59 Temple Place, Suite 330, Boston, MA 02111-1307 USA
    """

    # Create distance matrix unless given by parameters
    if inputdist == 'distance':
        D = x
        if init == 'default':
            init = 'cmdscale'
    else:
        D = cdist(x, x)
        if init == 'default':
            init = 'pca'

    if inputdist == 'distance' and init == 'pca':
        raise ValueError("Cannot use init == 'pca' when inputdist == 'distance'")

    if np.count_nonzero(np.diagonal(D)) > 0:
        raise ValueError("The diagonal of the dissimilarity matrix must be zero")

    # Remaining initialisation
    N = x.shape[0]
    scale = 0.5 / D.sum()
    D = D + np.eye(N)     

    if np.count_nonzero(D<=0) > 0:
        raise ValueError("Off-diagonal dissimilarities must be strictly positive")   

    Dinv = 1 / D
    if init == 'pca':
        [UU,DD,_] = np.linalg.svd(x)
        y = UU[:,:n]*DD[:n] 
    elif init == 'cmdscale':
        from cmdscale import cmdscale
        y,e = cmdscale(D)
        y = y[:,:n]
    else:
        y = np.random.normal(0.0,1.0,[N,n])
    one = np.ones([N,n])
    d = cdist(y,y) + np.eye(N)
    dinv = 1. / d
    delta = D-d 
    E = ((delta**2)*Dinv).sum() 

    # Get on with it
    for i in range(maxiter):

        # Compute gradient, Hessian and search direction (note it is actually
        # 1/4 of the gradient and Hessian, but the step size is just the ratio
        # of the gradient and the diagonal of the Hessian so it doesn't
        # matter).
        delta = dinv - Dinv
        deltaone = np.dot(delta,one)
        g = np.dot(delta,y) - (y * deltaone)
        dinv3 = dinv ** 3
        y2 = y ** 2
        H = np.dot(dinv3,y2) - deltaone - np.dot(2,y) * np.dot(dinv3,y) + y2 * np.dot(dinv3,one)
        s = -g.flatten(order='F') / np.abs(H.flatten(order='F'))
        y_old    = y

        # Use step-halving procedure to ensure progress is made
        for j in range(maxhalves):
            s_reshape = np.reshape(s, (-1,n),order='F')
            y = y_old + s_reshape
            d = cdist(y, y) + np.eye(N)
            dinv = 1 / d
            delta = D - d
            E_new = ((delta**2)*Dinv).sum()
            if E_new < E:
                break
            else:
                s = 0.5*s

        # Bomb out if too many halving steps are required
        if j == maxhalves-1:
            print('Warning: maxhalves exceeded. Sammon mapping may not converge...')

        # Evaluate termination criterion
        if abs((E - E_new) / E) < tolfun:
            if display:
                print('TolFun exceeded: Optimisation terminated')
            break

        # Report progress
        E = E_new
        if display > 1:
            print('epoch = %d : E = %12.10f'% (i+1, E * scale))

    if i == maxiter-1:
        print('Warning: maxiter exceeded. Sammon mapping may not have converged...')

    # Fiddle stress to match the original Sammon paper
    E = E * scale
    
    return [y,E]
###########################################################################################################################################################################
# Load in the data_train, time_values, and the MSLP raw data for the plotting

#You will need to open the data_train that we saved from the SOM Step #1
data_train = np.load('som_data_train.npy')
time_values = np.load('som_time_data.npy')
mslpraw = xr.open_dataset('SOM_MSLPraw_era_data.nc')
mslp = mslpraw['MSL'].values
mslp_SOM = mslpraw['MSL']
lon = mslpraw['lon'].values
lat = mslpraw['lat'].values
###########################################################################################################################################################################

In [5]:
#We will be making the Frequency and Sammon plots in this next section

#Set the SOM Columns and the Rows 
som_col = 5
som_row = 4
x_coor = 5
y_coor= 4
input_length = 1008

folderpath = '\\Users\\research\\thesis_code\\SOMs\\'  #This is where all your SOMs were saved

names = ([os.path.splitext(os.path.split(x)[-1])[0] for x in glob.glob("\\Users\\research\\thesis_code\\SOMs\\5by4_LR0.005_sig5_SOM_som_*.p")]) #this might be different for you

#but this is just grabbing the first few characters of my names of my file (see above how I named them, for example som_28

filepaths = glob.glob("\\Users\\research\\thesis_code\\SOMs\\5by4_LR0.005_sig5_SOM_som_*.p") #this is showing the path and the given file

In [ ]:
for path, name in zip(filepaths, names):
    with open (path, 'rb') as f:
        file = pickle.load(f) #This is loading every single som in that location
        frequencies = file.activation_response(data_train) #this is grabbing each freq
        q_error = round(file.quantization_error(data_train),4) #this is grabbing every q error out to 4 decimal places
        topo_error = round(file.topographic_error(data_train),4) #this is grabbing ever topographic error out to 4 decimal places
        plt.figure(figsize=(6,6))
        cs = plt.pcolormesh(frequencies, cmap='Blues')
        plt.title(name + ' ' + 'Frequencies,' + ' ' + 'Q_error =' + ' ' f"{q_error}" + ' ' + 'Topo_error =' + ' ' f"{topo_error}", fontsize=12)

        #in the title, I am plotting every q error and topo error from each som. You need to have the f" in front and whatever variable in {}
        #And this ' ' represents a space in the title
        plt.colorbar(cs)
        plt.ylim(4,0) # Change the 4 to whatever size SOM you have (this is the 2nd number)
        plt.tight_layout()
        plt.savefig(folderpath + 'frequencies_'+name+'.png') #I am saving the outputs as a png file in the same file path and giving it the name of each SOM
        
        plt.show()
        
        [y,E] = sammon(file.get_weights().reshape(som_col*som_row, input_length),2,display=1)

            # Plot Sammon map nodes
        plt.figure(figsize=(10,8))
        plt.scatter(y[:,0], y[:,1], s=20, c='black', marker='o')

            # Add lines between nodes
        mslp = np.reshape(y,(som_row,som_col,2))
        len_x, len_y, len_z = mslp.shape

        # add vertical lines
        for i in range(len_x-1):
            for j in range(len_y):
                plt.plot(mslp[i:i+2,j,0],mslp[i:i+2,j,1],c='black')

        # add horizontal lines
        for i in range(len_x):
            for j in range(len_y-1):
                plt.plot(mslp[i,j:j+2,0],mslp[i,j:j+2,1],c='black')  

        plt.xticks([])
        plt.yticks([])
        plt.title(name, fontsize=12)
        plt.savefig(folderpath + 'sammonplot_'+name+'.png') #I am saving the outputs as a png file in the same file path and giving it the name of each SOM

        plt.show()

In [ ]:
#This code Loop generates the Composite Plots for the SOM maps to ensure that the SOM generated Anoms. look like the composite mean photos in the raw data.
for path, name in zip(filepaths, names):
    with open (path, 'rb') as f:
        file = pickle.load(f)
        frequencies = file.activation_response(data_train)
        #print(frequencies)

        keys = [i for i in product(range(4), range(5))]
        winmap = {key: [] for key in keys}

        for i, x in enumerate(data_train):
            winmap[file.winner(x)].append(i)
        som_keys = getList(winmap)
        fig, axs = plt.subplots(4,5,
                        subplot_kw={'projection':ccrs.LambertConformal(central_longitude=-156, central_latitude=71, standard_parallels=(30, 60))},
                        figsize=(30, 15),facecolor='white') 

        for map_num in range(len(som_keys)):
            mslp_data = mslpraw[np.array(winmap[som_keys[map_num]])].mean(['time'])
            node = [som_keys[map_num][0],som_keys[map_num][1]]            #levs=0+(np.arange(990,1040,5))
            cs = axs[som_keys[map_num][0],som_keys[map_num][1]].contourf(lon, lat ,mslp_data, cmap='coolwarm',transform=ccrs.PlateCarree())
            axs[som_keys[map_num][0],som_keys[map_num][1]].set_extent([-131.7, -180, 73.48, 62.9], ccrs.PlateCarree() )
            axs[som_keys[map_num][0],som_keys[map_num][1]].set_title(f"Node : {node} sample size: {frequencies.flatten()[map_num]}", fontsize=8)

            axs[som_keys[map_num][0],som_keys[map_num][1]].coastlines()

            axs[som_keys[map_num][0],som_keys[map_num][1]].add_feature(cfeature.STATES)

            axs[som_keys[map_num][0],som_keys[map_num][1]].add_feature(cfeature.BORDERS)
        cbar_ax = fig.add_axes([0.08, 0.2, 0.5, 0.02])
        cbar=fig.colorbar(cs,cax=cbar_ax,orientation='horizontal')
        cbar.set_label('MSLP. (hPa)')
        plt.tight_layout()
        fig.subplots_adjust(bottom=0.25, top=0.9, left=0.05, right=0.6,wspace=0.05, hspace=0.25)
        plt.suptitle(name + ': MSLP Raw Data Plotted 4 by 5 Asymptotic Method [Sigma=3, LR=0.005]', x= 0.33 ,fontsize=22)

        #plt.savefig(folderpath + 'composites_'+name+'.png')
        plt.show()